# EDA Seguro Médico

Análisis exploratorio de datos del dataset de costos de seguros médicos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

DATA_DIR = '../data'
OUTPUT_DIR = '../output'
FIGS_DIR = os.path.join(OUTPUT_DIR, 'figs')
os.makedirs(FIGS_DIR, exist_ok=True)

df = pd.read_csv(os.path.join(DATA_DIR, 'insurance.csv'))
print(f'Dataset cargado: {df.shape[0]} registros, {df.shape[1]} columnas')
print(df.head())

## 2. Información del Dataset

Se muestra la información general del dataset, incluyendo el número de registros, columnas y tipos de datos.

In [ ]:
df.info()

## 3. Valores Faltantes

Se verifica la presencia de valores faltantes en el dataset.

In [ ]:
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else 'No hay valores faltantes ✓')

## 4. Estadísticas Descriptivas

Se calculan y muestran las estadísticas descriptivas para todas las columnas del dataset.

In [ ]:
df.describe(include='all').round(2)

## 5. Transformaciones Iniciales

Se convierten las columnas categóricas a tipo `category` para optimizar el uso de memoria y facilitar su análisis.

In [ ]:
cat_cols = ['sex', 'smoker', 'region']
for col in cat_cols:
    df[col] = df[col].astype('category')

print('✓ Columnas categóricas convertidas:', cat_cols)
print(df.dtypes)

## 6. Análisis Univariante - Variables Numéricas

Se visualiza la distribución de las variables numéricas para entender su comportamiento.

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f'Variables numéricas: {num_cols}\n')

for col in num_cols:
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    sns.histplot(df[col], kde=True, ax=ax, bins=30, color='steelblue')
    ax.set_title(f'Distribución de {col}', fontsize=12, fontweight='bold')
    ax.set_xlabel(col)
    ax.set_ylabel('Frecuencia')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'01_univariate_{col}.png'), dpi=100)
    plt.show()
    print(f'{col}: Min={df[col].min():.2f}, Max={df[col].max():.2f}, Media={df[col].mean():.2f}')

## 7. Análisis Univariante - Variables Categóricas

Se visualiza la distribución de las variables categóricas para entender su composición.

In [ ]:
for col in cat_cols + ['children']:
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    sns.countplot(data=df, x=col, ax=ax, palette='Set2')
    ax.set_title(f'Conteo por {col}', fontsize=12, fontweight='bold')
    ax.set_ylabel('Cantidad')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'02_univariate_{col}.png'), dpi=100)
    plt.show()
    print(f'\n{col}:')
    print(df[col].value_counts())

## 8. Filtrado de Outliers

Se eliminan los outliers de la variable `charges` utilizando el método del Rango Intercuartílico (IQR) para evitar que afecten el rendimiento del modelo.

In [ ]:
def remove_outliers_iqr(df_in, col, k=1.5):
    """Elimina outliers usando el método IQR."""
    q1 = df_in[col].quantile(0.25)
    q3 = df_in[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - k * iqr
    upper_bound = q3 + k * iqr
    
    print(f'Q1={q1:.2f}, Q3={q3:.2f}, IQR={iqr:.2f}')
    print(f'Límite inferior: {lower_bound:.2f}')
    print(f'Límite superior: {upper_bound:.2f}')
    
    mask = (df_in[col] >= lower_bound) & (df_in[col] <= upper_bound)
    outliers_removed = (~mask).sum()
    print(f'Outliers eliminados: {outliers_removed}')
    
    return df_in.loc[mask]

print(f'Dataset original: {df.shape[0]} registros')
df_filtered = remove_outliers_iqr(df, 'charges', k=1.5)
print(f'Dataset después de filtrar: {df_filtered.shape[0]} registros')
porcentaje_eliminado = (df.shape[0] - df_filtered.shape[0]) / df.shape[0] * 100
print(f'Registros eliminados: {df.shape[0] - df_filtered.shape[0]} ({porcentaje_eliminado:.1f}%)')

## 9. Tratamiento de la Variable Objetivo

Se analiza la distribución de la variable objetivo `charges` y se le aplica una transformación logarítmica para normalizar su distribución.

In [ ]:
print('=== ANÁLISIS DE CHARGES ===')
min_charges = df_filtered['charges'].min()
max_charges = df_filtered['charges'].max()
mean_charges = df_filtered['charges'].mean()
median_charges = df_filtered['charges'].median()

print(f'Mín: {min_charges:.2f}')
print(f'Máx: {max_charges:.2f}')
print(f'Media: {mean_charges:.2f}')
print(f'Mediana: {median_charges:.2f}')
print(f'Asimetría (Skewness): {df_filtered["charges"].skew():.2f}')
print(f'Curtosis: {df_filtered["charges"].kurtosis():.2f}')

if (df_filtered['charges'] > 0).all():
    df_filtered['log_charges'] = np.log(df_filtered['charges'])
    print('\n✓ Transformación logarítmica aplicada a charges')
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    sns.histplot(df_filtered['charges'], kde=True, ax=axes[0], color='steelblue', bins=30)
    axes[0].set_title('Charges - Original', fontweight='bold')
    axes[0].set_xlabel('Charges')
    
    sns.histplot(df_filtered['log_charges'], kde=True, ax=axes[1], color='coral', bins=30)
    axes[1].set_title('Charges - Log Transformado', fontweight='bold')
    axes[1].set_xlabel('Log(Charges)')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, '03_charges_transformation.png'), dpi=100)
    plt.show()
else:
    print('⚠ Advertencia: charges tiene valores ≤ 0')

## 10. Análisis Bivariante - Numéricas vs. Charges

Se analiza la relación entre las variables numéricas y la variable objetivo `charges`.

In [ ]:
numeric_features = [col for col in num_cols if col not in ['charges', 'log_charges']]

for col in numeric_features:
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    sns.scatterplot(data=df_filtered, x=col, y='charges', alpha=0.5, ax=ax, color='steelblue')
    
    corr = df_filtered[col].corr(df_filtered['charges'])
    
    ax.set_title(f'Charges vs {col} (Correlacion: {corr:.3f})', fontsize=12, fontweight='bold')
    ax.set_ylabel('Charges')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'04_bivariate_{col}_vs_charges.png'), dpi=100)
    plt.show()

## 11. Análisis Bivariante - Categóricas vs. Charges

Se analiza la relación entre las variables categóricas y la variable objetivo `charges`.

In [ ]:
for col in cat_cols + ['children']:
    fig, ax = plt.subplots(1, 1, figsize=(9, 5))
    sns.boxplot(data=df_filtered, x=col, y='charges', ax=ax, palette='Set2')
    ax.set_title(f'Distribucion de Charges por {col}', fontsize=12, fontweight='bold')
    ax.set_ylabel('Charges')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGS_DIR, f'05_bivariate_{col}.png'), dpi=100)
    plt.show()

## 12. Matriz de Correlación

Se calcula y visualiza la matriz de correlación entre las variables numéricas para identificar posibles relaciones lineales.

In [ ]:
corr_matrix = df_filtered.select_dtypes(include=[np.number]).corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Matriz de Correlacion - Variables Numericas', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(FIGS_DIR, '06_correlation_matrix.png'), dpi=100)
plt.show()

print('\n=== CORRELACIONES CON CHARGES ===')
correlations = corr_matrix['charges'].sort_values(ascending=False)
print(correlations)

## 13. División Train/Test Estratificado

Se divide el dataset en conjuntos de entrenamiento y prueba de forma estratificada utilizando la variable `charges` para asegurar que la distribución de la variable objetivo sea similar en ambos conjuntos.

In [ ]:
n_bins = 5
df_filtered['charges_bin'] = pd.qcut(df_filtered['charges'], q=n_bins, labels=False, duplicates='drop')

print(f'Bins creados: {df_filtered["charges_bin"].nunique()}')
print(f'\nDistribucion por bin:\n{df_filtered["charges_bin"].value_counts().sort_index()}')

train_df, test_df = train_test_split(
    df_filtered, 
    test_size=0.2, 
    random_state=42, 
    stratify=df_filtered['charges_bin']
)

train_df = train_df.drop(columns=['charges_bin', 'log_charges'])
test_df = test_df.drop(columns=['charges_bin', 'log_charges'])

train_pct = train_df.shape[0] / df_filtered.shape[0] * 100
test_pct = test_df.shape[0] / df_filtered.shape[0] * 100

print(f'\nTrain set: {train_df.shape[0]} registros ({train_pct:.1f}%)')
print(f'Test set: {test_df.shape[0]} registros ({test_pct:.1f}%)')

## 14. Verificación de Estratificación

Se compara la distribución de la variable `charges` en los conjuntos de entrenamiento y prueba para verificar que la estratificación se haya realizado correctamente.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

sns.histplot(train_df['charges'], kde=True, ax=axes[0], bins=30, color='steelblue')
axes[0].set_title('Train - Distribucion de Charges', fontweight='bold')
axes[0].set_xlabel('Charges')

sns.histplot(test_df['charges'], kde=True, ax=axes[1], bins=30, color='coral')
axes[1].set_title('Test - Distribucion de Charges', fontweight='bold')
axes[1].set_xlabel('Charges')

plt.tight_layout()
plt.savefig(os.path.join(FIGS_DIR, '07_train_test_distribution.png'), dpi=100)
plt.show()

print('=== ESTADÍSTICAS COMPARATIVAS ===')
train_mean = train_df['charges'].mean()
train_std = train_df['charges'].std()
test_mean = test_df['charges'].mean()
test_std = test_df['charges'].std()
diff_means = abs(train_mean - test_mean)

print(f'Train - Media: {train_mean:.2f}, Desv Std: {train_std:.2f}')
print(f'Test  - Media: {test_mean:.2f}, Desv Std: {test_std:.2f}')
print(f'\nDiferencia de medias: {diff_means:.2f}')

## 15. Validación de Datos Limpios

Se realiza una última validación de los conjuntos de entrenamiento y prueba para asegurar que no contengan valores faltantes ni duplicados.

In [ ]:
print('=== VALIDACIÓN TRAIN ===')
print(f'Registros: {train_df.shape[0]}')
print(f'Valores faltantes: {train_df.isnull().sum().sum()}')
print(f'Duplicados: {train_df.duplicated().sum()}')

print('\n=== VALIDACIÓN TEST ===')
print(f'Registros: {test_df.shape[0]}')
print(f'Valores faltantes: {test_df.isnull().sum().sum()}')
print(f'Duplicados: {test_df.duplicated().sum()}')

print('\n✓ Datasets listos para modelado')

## 16. Guardado de Archivos

Se guardan los datasets de entrenamiento y prueba, así como los resultados del análisis exploratorio en archivos de texto.

In [ ]:
train_df.to_csv(os.path.join(DATA_DIR, 'train.csv'), index=False)
test_df.to_csv(os.path.join(DATA_DIR, 'test.csv'), index=False)
print('✓ Archivos salvados: train.csv, test.csv')

with open(os.path.join(OUTPUT_DIR, 'deductions.txt'), 'w', encoding='utf-8') as f:
    f.write('=== DEDUCCIONES DEL EDA ===\n\n')
    f.write('UNIVARIANTE:\n')
    f.write('- charges es fuertemente sesgada a la derecha (outliers en costos altos)\n')
    f.write('- smoker tiene desequilibrio: 20% fuma, 80% no fuma\n')
    f.write('- edad distribuida uniformemente (18-64 años)\n')
    f.write('- region uniforme entre 4 categorias\n\n')
    f.write('BIVARIANTE:\n')
    f.write('- Fumadores pagan 3-4x mas que no fumadores (VARIABLE MAS IMPORTANTE)\n')
    f.write('- age: correlacion fuerte (0.60) - relacion cuadratica\n')
    f.write('- bmi: correlacion moderada (0.41)\n')
    f.write('- sex y region: correlaciones muy debiles (<0.1)\n')
    f.write('- children: correlacion casi nula (-0.08)\n\n')
    f.write('ACCIONES TOMADAS:\n')
    f.write(f'- Eliminados {df.shape[0] - df_filtered.shape[0]} outliers por IQR\n')
    f.write('- Aplicada transformacion logaritmica a charges\n')
    f.write('- Split 80/20 estratificado por charges\n')

print('✓ Deducciones guardadas: deductions.txt')

with open(os.path.join(OUTPUT_DIR, 'correlations.txt'), 'w', encoding='utf-8') as f:
    f.write('=== CORRELACIONES CON CHARGES (VARIABLE OBJETIVO) ===\n\n')
    for var, corr_val in correlations.items():
        f.write(f'{var}: {corr_val:.4f}\n')

print('✓ Correlaciones guardadas: correlations.txt')

with open(os.path.join(OUTPUT_DIR, 'summary.txt'), 'w', encoding='utf-8') as f:
    f.write('=== RESUMEN EDA SEGURO MEDICO ===\n\n')
    f.write('DATASET ORIGINAL:\n')
    f.write(f'- Registros: {df.shape[0]}\n')
    f.write(f'- Caracteristicas: {df.shape[1]}\n')
    f.write(f'- Valores faltantes: 0\n\n')
    f.write('DESPUES DE FILTRADO (IQR k=1.5):\n')
    f.write(f'- Registros: {df_filtered.shape[0]}\n')
    f.write(f'- Outliers removidos: {df.shape[0] - df_filtered.shape[0]}\n')
    pct_removed = (df.shape[0] - df_filtered.shape[0]) / df.shape[0] * 100
    f.write(f'- Porcentaje eliminado: {pct_removed:.2f}%\n\n')
    f.write('SPLIT FINAL:\n')
    f.write(f'- Train: {train_df.shape[0]} registros (80%)\n')
    f.write(f'- Test: {test_df.shape[0]} registros (20%)\n\n')
    f.write('VARIABLES NUMERICAS:\n')
    f.write(f'- age: rango 18-64 años\n')
    f.write(f'- bmi: rango 15.96-53.13\n')
    train_min_charges = train_df['charges'].min()
    train_max_charges = train_df['charges'].max()
    f.write(f'- charges: rango {train_min_charges:.2f} - {train_max_charges:.2f}\n\n')
    f.write('VARIABLES CATEGORICAS:\n')
    f.write(f'- sex: male, female\n')
    f.write(f'- smoker: yes (fumadores), no\n')
    f.write(f'- region: southwest, southeast, northwest, northeast\n')
    f.write(f'- children: 0 a 5 hijos\n')

print('✓ Resumen guardado: summary.txt')

print('\n=== EDA COMPLETADO EXITOSAMENTE ===')
print('\n📁 Archivos generados:')
print(f'   - train.csv ({train_df.shape[0]} registros)')
print(f'   - test.csv ({test_df.shape[0]} registros)')
print('   - deductions.txt')
print('   - correlations.txt')
print('   - summary.txt')
print('   - 7 graficas PNG en output/figs/')